LSTM Нейросеть - Игорь

In [1]:
# Для нейросети
from config import Config
from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.models import Sequential # Полносвязная модель
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Flatten, Activation, Embedding, SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
%matplotlib inline

In [2]:
trainWordIndexesFilename = "Копия trainWordIndexes.npy"
testWordIndexesFilename = "Копия testWordIndexes.npy"

trainWordIndexes = np.load(Config.TRAIN_WORD_INDEXES_FILENAME, allow_pickle=True)
testWordIndexes = np.load(Config.TEST_WORD_INDEXES_FILENAME, allow_pickle=True)

In [3]:
def getSetFromIndexes(wordIndexes, xLen, step):
  xText = []
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen): # Идём по всей длине вектора индексов
    xText.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаемся вперёд на step
    
  return xText


# Формирование обучающей и проверочной выборки для каждого класса
# wordIndexes - массив индексов
# xLen - размер окна
# step - шаг окна

def createSetsMultiClasses(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна
  nClasses = len(wordIndexes) # Количество классов
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна"
  for wI in wordIndexes:      # Для каждого текста выборки из последовательности индексов
    classesXSamples.append(getSetFromIndexes(wI, xLen, step))

  # Формируем один общий xSamples
  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)): # Перебираем каждое окно определенного класса
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [4]:
xLen = 50 # Размер окна (количество слов в векторе)
step = 6 # Шаг разбиения текста на векторы

In [5]:
xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, xLen)
xVal, yVal = xTrain[:10000], yTrain[:10000]
xTrain, yTrain = xTrain[10000:], yTrain[10000:]
xBuffer, yBuffer = xTest, yTest
xTest, yTest = xVal, yVal
xVal, yVal = xBuffer, yBuffer
# для каждого "окна" класса мы в соответствие ставим значение true или false => поэтому длины x и y одинаковые
print("Размерности тренировочного набора")
print(xTrain.shape)
print(yTrain.shape)
print()
print("Размерности тестового набора")
print(xTest.shape)
print(yTest.shape)
print()
print("Размерности val набора")
print(xVal.shape)
print(yVal.shape)

Размерности тренировочного набора
(57354, 50)
(57354, 2)

Размерности тестового набора
(10000, 50)
(10000, 2)

Размерности val набора
(872, 50)
(872, 2)


In [6]:
numWords = 20000

In [7]:
modelLSTM = Sequential()

modelLSTM.add(Embedding(numWords, 100, input_length=xLen))
modelLSTM.add(SpatialDropout1D(0.2))
modelLSTM.add(BatchNormalization())

modelLSTM.add(LSTM(10))
modelLSTM.add(Dense(2, activation='softmax'))

modelLSTM.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

modelLSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           2000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 100)          0         
 ropout1D)                                                       
                                                                 
 batch_normalization (BatchN  (None, 50, 100)          400       
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 10)                4440      
                                                                 
 dense (Dense)               (None, 2)                 22        
                                                                 
Total params: 2,004,862
Trainable params: 2,004,662
Non-

In [8]:
# Обучаем сеть
Batch_Size = 256

historyLSTM = modelLSTM.fit(xTrain, 
                            yTrain, 
                            epochs=15,
                            batch_size=Batch_Size,
                            # validation_data=(xVal, yVal),
                            use_multiprocessing=True)

Epoch 1/15


2021-12-15 00:54:22.396292: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


225/225 [==============================] - 10s 41ms/step - loss: 0.3770 - accuracy: 0.8435
Epoch 2/15
225/225 [==============================] - 9s 40ms/step - loss: 0.0633 - accuracy: 0.9829
Epoch 3/15
225/225 [==============================] - 9s 40ms/step - loss: 0.0193 - accuracy: 0.9952
Epoch 4/15
225/225 [==============================] - 9s 42ms/step - loss: 0.0078 - accuracy: 0.9985
Epoch 5/15
225/225 [==============================] - 10s 42ms/step - loss: 0.0048 - accuracy: 0.9991
Epoch 6/15
225/225 [==============================] - 10s 44ms/step - loss: 0.0041 - accuracy: 0.9992
Epoch 7/15
225/225 [==============================] - 11s 48ms/step - loss: 0.0032 - accuracy: 0.9994
Epoch 8/15
225/225 [==============================] - 10s 46ms/step - loss: 0.0016 - accuracy: 0.9997
Epoch 9/15
225/225 [==============================] - 10s 46ms/step - loss: 0.0028 - accuracy: 0.9994
Epoch 10/15
225/225 [==============================] - 10s 46ms/step - loss: 0.0019 - accuracy: 